In [ ]:
print("Hallo")

Hallo


We will learn how to load pre-trained models from Hugging Face and make inferences using the Pipeline module. Additionally, we will learn how to further train pre-trained LLMs on your own data (self-supervised fine-tuning).  We will have a solid understanding of how to pretrain LLMs and store them to later fine-tune for  specific use cases. This will empower  to create powerful and customized natural language processing solutions.

In [2]:
!pip install --user datasets # 2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Pretraining and Self supervised learning

Pretraining is a technique used in natural language processing (NLP) to train large language models (LLMs) on a vast corpus of unlabeled text data. The goal is to capture the general patterns and semantic relationships present in natural language, allowing the model to develop a deep understanding of language structure and meaning.

The motivation behind pretraining transformers is to address the limitations of traditional NLP approaches that often require significant amounts of labeled data for each specific task. By leveraging the abundance of unlabeled text data, pretraining enables the model to learn fundamental language skills through self-supervised objectives, facilitating transfer learning.

The pretraining objectives, such as masked language modeling (MLM) and next sentence prediction (NSP), play a crucial role in the success of transformer models. Pretrained models can be further tuned by training them on domain-specific unlabeled data, which is known as self-supervised fine-tuning.

Also, the model can be fine-tuned on specific downstream tasks using labeled data, a process known as supervised fine-tuning, further improving its performance.


In [5]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
print(pipe("This movie was really")[0]["generated_text"])

Device set to use cpu


This movie was really good. I was really surprised by how good it was.
I was
